# BEE 4750 Final Project: Solar Photovoltaic Feasibility Study on Cornell's Engineering Quadrangle

**Names**: Sitara Sastry, Maya Zoe Yu

**ID**: sls496, mzy3

> **Introduction**
>
> The following model will conduct a feasibility study of solar energy production on Cornell's Engineering Quadrangle. As Cornell’s Climate Action Plan seeks to have Cornell’s Ithaca campus carbon neutral by 2035 through utilizing 100% renewable energy, the proposed research will analyze if solar energy would help to achieve these goals. As such, the study will analyze how Cornell can invest in solar photovoltaics (PV) to increase this percentage – focusing mainly on the Engineering Quadrangle. In addition, the model will analyze the theoretical production of solar rooftop PV systems while optimzing cost. 


In [7]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("GraphRecipes")
Pkg.add("Plots")
Pkg.add("Measures")
Pkg.add("MarkdownTables")

  Activating project at `~/BEE 4750/bee4750_solar`


   Resolving package versions...


  No Changes to `~/BEE 4750/bee4750_solar/Project.toml`
  No Changes to `~/BEE 4750/bee4750_solar/Manifest.toml`


   Resolving package versions...


  No Changes to `~/BEE 4750/bee4750_solar/Project.toml`
  No Changes to `~/BEE 4750/bee4750_solar/Manifest.toml`


...

  No Changes to `~/BEE 4750/bee4750_solar/Project.toml`
  No Changes to `~/BEE 4750/bee4750_solar/Manifest.toml`


   Resolving package versions...


  No Changes to `~/BEE 4750/bee4750_solar/Project.toml`
  No Changes to `~/BEE 4750/bee4750_solar/Manifest.toml`


   Resolving package versions...


  No Changes to `~/BEE 4750/bee4750_solar/Project.toml`
  No Changes to `~/BEE 4750/bee4750_solar/Manifest.toml`


### Theoretical Solar Output Model

To determine the theoretical solar output, we must calculate the POA (plane-of-array) irradiance that is incident to the solar PV system. The equations utilized in this section were derived from Energy Systems Engineering: Evaluation and Implementation, Fourth Edition by Vanek et al., 2021. 

We will first determine the declination angle ($\delta$) – the angular distance in degrees between the sun's rays and the plane of the Earth's equator. We may determine $\delta$ as follows: 
- $\delta = 23.45 sin[\frac{360(284+N)}{365}]$ where $N$ is the Julian date

Then, we determine the sunset hour angle ($\omega_s$) which is the angular distance in which the sun has an altitudee angle of 0° and is incident with the horizon. Between the fall and spring equinox, the sunset hour angle may be calculated as: 
- $\omega_s = acos[-tan(L)tan(\delta)]$ where L is the latitude (42.4°)

However, during the spring and fall equinox, the sun may strike the back of the device. Therefore, we must adjust our equation for the sunset hour angle as follows: 
- $\omega_s' = acos[-tan(L-\beta)tan(\delta)]$ where $\beta$ is the tilt angle of the array ($\beta = 10°$)

Next, we calculate the ratio of diffuse insolation to global insolation ($\frac{H_d}{H}$) as follows: 
- $\frac{H_d}{H} = 1.39-4.03K_t+5.53K_t^2-3.11K_t^3$ where $K_t$ is the clearness index 

We may then calculate the ratio of direct insolation at an angle to the direct insolation onto a flat, horizontal surface as follows: 
- $R_b = \frac{cos(L-\beta)cos(\delta)sin(\omega_s')+\omega_s'sin(L-\beta)sin(\delta)}{cos(L)cos(\delta)sin(\omega_s')+\omega_s'sin(L)sin(\delta)}$

Now, we calculate the direct, diffuse, and reflected components of insolation as follows: 
- Direct = $(1-\frac{H_d}{H})R_b$
- Diffuse = $\frac{H_d}{H}(\frac{1+cos(\beta)}{2}$
- Reflected = $0.2[\frac{1-cos(\beta)}{2}]$

In [8]:
### Solar Insolation - Direct, Diffuse, Reflected
function solar_insolation(N, L, β, K_t, ρ, H)
    # Convert relevant inputs into radians 
    L = deg2rad(L)
    β = deg2rad(β)

    # Calculate ratio of diffuse insolation to global insolation
    ratio_diffuse_global = 1.39-4.03(K_t)+5.53(K_t)^2-3.11(K_t)^3 

    # Calculate sunset hour angle based on N
    δ = 23.45(sin(deg2rad(360(284+N))/365)) # Declination Angle
    δ = deg2rad(δ)
    if N >= 79 && N <= 265
        ω_s = min(acos(-tan(L)*tan(δ)), acos(-tan(L-β)*tan(δ)))
    else 
        ω_s = acos(-tan(L)*tan(δ))
    end

    # Ratio of Beam at Angle to Beam onto Horizontal Surface
    R_b=(cos(L-β)*cos(δ)*sin(ω_s)+ω_s*sin(L-β)sin(δ))/(cos(L)*cos(δ)*sin(ω_s)+ω_s*sin(L)*sin(δ))

    # Calculate direct, diffuse, and reflected insolation
    Direct = (1-ratio_diffuse_global)*R_b
    Diffuse = (ratio_diffuse_global)*(1+cos(β))/2
    Reflected = ρ*(1-cos(β))/2

    # Calcuate ratio of global insolation at angle to horizontal
    Incident = Direct+Diffuse+Reflected
    
    # Calculate total insolation onto angled surface at angle β [Energy/Area]
    H_t = H*Incident

    # Return all outputs 
    ω_s = rad2deg(ω_s)
    return H_t
end


# Define relevant parameter inputs (https://www.nrel.gov/docs/legosti/old/789.pdf)
L = 42.4
β = 10
N = [i for i = 1:365]
K_t = zeros(365)
K_t[1:31] .= 0.312
K_t[32:59] .= 0.337
K_t[60:90] .= 0.364
K_t[91:120] .= 0.411
K_t[121:151] .= 0.428
K_t[152:181] .= 0.454
K_t[182:212] .= 0.460
K_t[213:243] .= 0.442
K_t[244:273] .= 0.429
K_t[274:304] .= 0.397
K_t[305:334] .= 0.299 
K_t[335:365] .= 0.269
H = zeros(365)
H[1:31] .= 4378
H[32:59] .= 6535
H[60:90] .= 9774
H[91:120] .= 14091
H[121:151] .= 16977
H[152:181] .= 19082
H[182:212] .= 18830
H[213:243] .= 16168
H[244:273] .= 12837
H[274:304] .= 8843
H[305:334] .= 4696
H[335:365] .= 3373
H = H ./ 3600 # Convert to kWh

# Calculate insolation based on relevant parameter inputs 
insolation = zeros(365)
for i in 1:365
    day = N[i]
    clearness_index = K_t[i]
    global_insolation = H[i]
    insolation[i] = solar_insolation(day, L, β, clearness_index, 0.2, global_insolation) # kWh/m^2
end 


### Analysis of Electrical Demand and Rooftop Area Availability 

Cornell's Engineering Quadrangle encompasses the following buildings: Bard, Carpenter, Duffield, Hollister, Kimball, Olin, Phillips, Snee, Rhodes, Upson, and Ward . The yearly electricity demand and available rooftop area for these buildings may be found through the following:

In [9]:
using CSV
using DataFrames

### Annual Energy Demand
bard = DataFrame(CSV.File("BardElectricData.csv"));
bardDemand = sum(bard.Data[1:end]);
carpenter = DataFrame(CSV.File("CarpenterElectricData.csv"));
carpenterDemand = sum(carpenter.Data[1:end]);
duffield = DataFrame(CSV.File("DuffieldElectricData.csv"));
duffieldDemand = sum(duffield.Data[1:end]);
hollister = DataFrame(CSV.File("HollisterElectricData.csv"));
hollisterDemand = sum(hollister.Data[1:end]);
kimball = DataFrame(CSV.File("KimballElectricData.csv"));
kimballDemand = sum(kimball.Data[1:end]);
olin = DataFrame(CSV.File("OlinElectricData.csv"));
olinDemand = sum(olin.Data[1:end]);
phillips = DataFrame(CSV.File("PhillipsElectricData.csv"));
phillipsDemand = sum(phillips.Data[1:end]);
snee = DataFrame(CSV.File("SneeElectricData.csv"));
sneeDemand = sum(snee.Data[1:end]);
rhodes = DataFrame(CSV.File("RhodeElectricData.csv"));
rhodesDemand = sum(rhodes.Data[1:end]);
upson = DataFrame(CSV.File("UpsonElectricData.csv"));
upsonDemand = sum(upson.Data[1:end]);
ward = DataFrame(CSV.File("WardElectricData.csv"));
wardDemand = sum(ward.Data[1:end]);

### Rooftop Area (in meters)
bardArea = 1060.34;
carpenterArea = 1670.21;
duffieldArea = 1971.78;
hollisterArea = 1965.75;
kimballArea = 956.47;
olinArea = 1529.58;
phillipsArea = 1665.13;
sneeArea = 1680.23;
rhodesArea = 3824.18;
upsonArea = 2176.19;
wardArea = 1033.94;

### Create dataframe for values 
Buildings = ["Bard", "Carpenter", "Duffield", "Hollister", "Kimball", "Olin", "Phillips", "Snee", "Rhodes", "Upson", "Ward"]
Demand = [bardDemand, carpenterDemand, duffieldDemand, hollisterDemand, kimballDemand, olinDemand, phillipsDemand, sneeDemand, rhodesDemand, upsonDemand, wardDemand]
Area = [bardArea, carpenterArea, duffieldArea, hollisterArea, kimballArea, olinArea, phillipsArea, sneeArea, rhodesArea, upsonArea, wardArea]
Available_Area_Solar = Area .* 0.75
EngQuad = DataFrame(Buildings = Buildings, Annual_Demand_kWh = Demand, Rooftop_Area = Area, Available_Area_Solar = Available_Area_Solar)

Row,Buildings,Annual_Demand_kWh,Rooftop_Area,Available_Area_Solar
,String,Float64,Float64,Float64
1,Bard,30472.8,1060.34,795.255
2,Carpenter,15508.6,1670.21,1252.66
3,Duffield,333403.0,1971.78,1478.84
4,Hollister,41875.3,1965.75,1474.31
5,Kimball,121797.0,956.47,717.352
6,Olin,91384.0,1529.58,1147.18
7,Phillips,10857.3,1665.13,1248.85
8,Snee,43301.0,1680.23,1260.17
9,Rhodes,40800.7,3824.18,2868.13


### Optimization for Electricity Demand and Area (RSD)

In [18]:
using JuMP
using HiGHS
### defining model
economic_model = Model(HiGHS.Optimizer);

### costs
module_cost = 2900; # units $ per kW
inverter_cost = 180; # units $ per kW
RSD_cost = 62; # units $ per module

### defining variables
K = 1:11;
M = 1000;
total_insolation = sum(insolation);

### assigning decision variables
@variable(economic_model, capacity[k in K] >= 0);
@variable(economic_model, num_modules[k in K] >= 0, Int);

### defining constraints
@constraint(economic_model, rooftop_area_min[k in K], num_modules[k].*2.17 >= 0.20.*Available_Area_Solar[k]);
@constraint(economic_model, rooftop_area_max[k in K], num_modules[k]*2.17 <= Available_Area_Solar[k]);
@constraint(economic_model, total_production, sum(0.207*num_modules[:]*2.17*total_insolation) >= 0.50*sum(Demand[:]))
@constraint(economic_model, capacity_const[k in K], capacity[k] == 0.450*num_modules[k]);

### assigining objective function
@objective(economic_model, Min, sum(capacity[k]*module_cost for k in K)+sum(capacity[k]*inverter_cost for k in K)+sum(num_modules[k]*RSD_cost for k in K))

### optimizing the model
optimize!(economic_model)
display(value.(capacity))
display(value.(num_modules))

### create data table 
Buildings = ["Bard", "Carpenter", "Duffield", "Hollister", "Kimball", "Olin", "Phillips", "Snee", "Rhodes", "Upson", "Ward"]
capacity = [33.30, 52.20, 61.65, 61.20, 30.15, 47.70, 52.20, 52.65, 119.25, 67.95, 32.4]
modules = [74, 116, 137, 136, 67, 106, 116, 117, 265, 151, 72]
RSD = DataFrame(Buildings = Buildings, Installed_Capacity = capacity, Modules = modules)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:11
And data, a 11-element Vector{Float64}:
  33.300000000000004
  52.2
  61.65
  61.2
  30.150000000000002
  47.7
  52.2
  52.65
 119.25
  67.95
  32.4

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:11
And data, a 11-element Vector{Float64}:
  74.0
 116.0
 137.0
 136.0
  67.0
 106.0
 116.0
 117.0
 265.0
 151.0
  72.0

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      1964936
  Dual bound        1964936
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    1964936 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


Row,Buildings,Installed_Capacity,Modules
,String,Float64,Int64
1,Bard,33.3,74
2,Carpenter,52.2,116
3,Duffield,61.65,137
4,Hollister,61.2,136
5,Kimball,30.15,67
6,Olin,47.7,106
7,Phillips,52.2,116
8,Snee,52.65,117
9,Rhodes,119.25,265


### Optimization for Electricity Demand and Area (UL 3741)

We may report our problem in standard form as follows: 

$$\begin{equation}
\begin{aligned}
& \min_{x, y} & \sum\limits_{k ∈ K} MC(x_k)+\sum\limits_{k ∈ K} IC(x_k)+\sum\limits_{k ∈ K} RSD(y_k)\\
&\text{subject to} & \\
& & y_k(2.17) ≥ 0.20(A_k) \qquad∀k ∈ K\\
& & y_k(2.17) ≤ (A_k) \qquad∀k ∈ K\\
& & \sum\limits_{k ∈ K} 0.207(y_k)(2.17)(I) ≥ 0.50 \sum\limits_{k ∈ K} Demand_k\\
& & x_k = y_k(0.450) \qquad∀k ∈ K\\
\end{aligned}
\end{equation}$$

In [13]:
### defining model
economic_model_ul3741 = Model(HiGHS.Optimizer);

### costs
module_cost = 2900; # units $ per kW
inverter_cost = 250; # units $ per kW

### defining variables
K = 1:11;
M = 1000;
total_insolation = sum(insolation);

### assigning decision variables
@variable(economic_model_ul3741, capacity[k in K] >= 0);
@variable(economic_model_ul3741, num_modules[k in K] >= 0, Int);

### defining constraints
@constraint(economic_model_ul3741, rooftop_area_min[k in K], num_modules[k].*2.17 >= 0.20.*Available_Area_Solar[k]);
@constraint(economic_model_ul3741, rooftop_area_max[k in K], num_modules[k]*2.17 <= Available_Area_Solar[k]);
@constraint(economic_model_ul3741, total_production, sum(0.207*num_modules[:]*2.17*total_insolation) >= 0.50*sum(Demand[:]))
@constraint(economic_model_ul3741, capacity_const[k in K], capacity[k] == 0.450*num_modules[k]);

### assigining objective function
@objective(economic_model_ul3741, Min, sum(capacity[k]*module_cost for k in K)+sum(capacity[k]*inverter_cost for k in K))

### optimizing the model
optimize!(economic_model_ul3741)
display(value.(capacity))
display(value.(num_modules))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:11
And data, a 11-element Vector{Float64}:
  33.300000000000004
  52.2
  61.65
  61.2
  30.150000000000002
  47.7
  52.2
  52.65
 119.25
  67.95
  32.4

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:11
And data, a 11-element Vector{Float64}:
  74.0
 116.0
 137.0
 136.0
  67.0
 106.0
 116.0
 117.0
 265.0
 151.0
  72.0

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      1923547.5
  Dual bound        1923547.5
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    1923547.5 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


#### Future Steps 
Result: If we start to change some assumptions (cost, area), how does it change? Does it change which we prefer? 
Reoptimize with different assumptions, or do robustness checking (re-evaluate what costs look like if things change? Point at which things switch)

Capacity constraint change, total production -> Shadow price 
How interpretable are our constraints? What if these were to change? 